In [221]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## this for package import and data prepare

In [222]:

import utlis_2nd.cusdom as custom
import super_learn_task_expr1 as expr1
import warnings

read_abso_path="/Users/liangaoming/Documents/Ai_station/neural_find_sol/wgan_2nd/complex_center_dataset/combined_data.pt"
warnings.filterwarnings("ignore")

from collections import namedtuple
# 定义 Transition
Transition = namedtuple("Transition", ["state", "action", "reward", "next_state", "done"])
# 创建一个 Transition 对象
trans = Transition(state=1, action=2, reward=3, next_state=4, done=False)


In [223]:
# split data & get the t_steps
train_loader,valid_loader,test_loader,yaml_config =\
    custom.return_train_valid_test4loader(abso_path=read_abso_path)
# look（shape）
for batch_idx, (data, target) in enumerate(train_loader):
    print("shape:", data.shape)
    print("label for csv:",target.shape)
    break


load data & read yaml
shape: torch.Size([256, 100, 9])
label for csv: torch.Size([256])


# Define the omega_neural network from nn_base

## Here: we just to evaluate the
### 1.omega_neural network
input_shape: the shape means that we have 100 time steps and 2 variables input **[-1,100,2]**
output_shape: output **[-1,51,2]**
### 2.inference_neural network
input_shape: the shape means that we have 100 time steps and 2 variables input **[-1，100，2]**
output_shape: output **[-1,100,2]**



In [224]:
results_path= "/Users/liangaoming/Documents/Ai_station/neural_find_sol/wgan_2nd/expr1/expr1_results/expr1_"
expr1.config["train_nomin"]= int((yaml_config['train_size_percent']*yaml_config['all_solus_numbers'])/yaml_config["batch_size"])
expr1.config["valid_nomin"]= int((yaml_config['valid_size_percent']*yaml_config['all_solus_numbers'])/yaml_config["batch_size"])
expr1.config["test_nomin"]= int((yaml_config['test_size_percent']*yaml_config['all_solus_numbers'])/yaml_config["batch_size"])
expr1.config["train_loader"]= train_loader
expr1.config["valid_loader"]= valid_loader
expr1.config["test_loader"]= test_loader
expr1.config["data_length"]= yaml_config["data_length"]
expr1.config["device"] = "cpu"
expr1.config["hidden_act"]= "rational"
expr1.config["inference_output_act"]="identity"
expr1.config["omega_output_act"]="softmax"
expr1.config["vari_number"]= yaml_config["vari_number"]
expr1.config["'sample_vesting"]=2 #2s
expr1.config["Omega_num_epoch"]=100

In [226]:
# return test' results
mse,u,mae,train_model=expr1.do_expr(results_save_path=results_path,
                                    folder_num=97,
                                    train_config=expr1.config,
                                    model_type="omega_net")

the prior knowledge is {'basis_1': 'x**0', 'basis_2': 'sin', 'basis_3': 'cos'}
test_only_omega
start train the omega neural network
***epoch 0
loss0.8200402181023246_epoch0---epoch_time2.4162399768829346
***epoch 1
loss0.09261259604773957_epoch1---epoch_time2.1675021648406982
***epoch 2
loss0.038453865099780216_epoch2---epoch_time2.2225520610809326
***epoch 3
loss0.01901188463469936_epoch3---epoch_time2.140362024307251
***epoch 4
loss0.011563584657844093_epoch4---epoch_time2.1347949504852295
***epoch 5
loss0.008425225514342126_epoch5---epoch_time2.1958112716674805
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/liangaoming/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/hg/r_268m7j71l0nb6gfjls2b900000gn/T/ipykernel_43493/2736944302.py", line 2, in <module>
    mse,u,mae,train_model=expr1.do_expr(results_save_path=results_path,
  File "/Users/liangaoming/Documents/Ai_station/neural_find_sol/wgan_2nd/expr1/super_learn_task_expr1.py", line 298, in do_expr
    return expr(expr_config=train_config,train_type=model_type)
  File "/Users/liangaoming/Documents/Ai_station/neural_find_sol/wgan_2nd/expr1/super_learn_task_expr1.py", line 252, in expr
    train_omega_actor(co_train_actor,config_device=expr_config["device"])
  File "/Users/liangaoming/Documents/Ai_station/neural_find_sol/wgan_2nd/expr1/super_learn_task_expr1.py", line 163, in train_omega_actor
    eval_mse_value,eval_u_stat,eval_mae=co_train_actor.train_omega_neural(dev


## here is a plot function to output a figure


In [206]:
import matplotlib.pyplot as plt
import os
import numpy as np

In [207]:
def plot_loss(path, loss_train, x_log=False):

    """
    Plot loss.
    Args:
        path (str): path.
        loss_train (list): list of training loss.
        x_log (bool): whether to use log scale for x-axis.
    Returns:
        None.
    """
    plt.figure()
    ax = plt.gca()
    y2 = np.asarray(loss_train)
    plt.plot(y2, 'k-', label='Train')
    plt.xlabel('epoch', fontsize=18)
    ax.tick_params(labelsize=18)
    plt.yscale('log')
    if x_log == False:
        fntmp = os.path.join(path, 'loss.png')

    else:
        plt.xscale('log')
        fntmp = os.path.join(path, 'loss_log.png')
    plt.tight_layout()
    plt.savefig(fntmp,dpi=300)

    plt.close()
def plot_model_output(path, args, output, epoch):

    plt.figure()
    ax = plt.gca()

    plt.plot(args.training_input.detach().cpu().numpy(),
             args.training_target.detach().cpu().numpy(), 'b*', label='True')
    plt.plot(args.test_input.detach().cpu().numpy(),
             output.detach().cpu().numpy(), 'r-', label='Test')

    ax.tick_params(labelsize=18)
    plt.legend(fontsize=18)
    fntmp = os.path.join(path, 'output', str(epoch)+'.png')

    plt.savefig(fntmp, dpi=300)


    plt.close()
def my_fft(data, freq_len=40, isnorm=1):

    """
    This function performs FFT on the given data.

    Args:
    data (numpy.ndarray): The input data.
    freq_len (int): The length of the frequency.
    isnorm (int): The normalization factor.

    Returns:
    return_fft (numpy.ndarray): The FFT output array.
    """

    # second_diff_input = np.mean(np.diff(np.diff(np.squeeze(x_input))))
    # if abs(second_diff_input) < 1e-10:
    datat = np.squeeze(data)
    datat_fft = np.fft.fft(datat)
    ind2 = range(freq_len)
    fft_coe = datat_fft[ind2]
    if isnorm == 1:
        return_fft = np.absolute(fft_coe)
    else:
        return_fft = fft_coe

    return return_fft
def SelectPeakIndex(FFT_Data, endpoint=True):

    """
    This function selects the peak index from FFT data.

    Args:
    FFT_Data (numpy.ndarray): The FFT data array.
    endpoint (bool): Whether to include endpoints or not. Default is True.

    Returns:
    sel_ind (numpy.ndarray): Selected index array with peaks.
    """

    D1 = FFT_Data[1:-1]-FFT_Data[0:-2]
    D2 = FFT_Data[1:-1]-FFT_Data[2:]
    D3 = np.logical_and(D1 > 0, D2 > 0)
    tmp = np.where(D3 == True)
    sel_ind = tmp[0]+1
    if endpoint: #
        if FFT_Data[0]-FFT_Data[1] > 0:
            sel_ind = np.concatenate([[0], sel_ind])
        if FFT_Data[-1]-FFT_Data[-2] > 0:
            Last_ind = len(FFT_Data)-1
            sel_ind = np.concatenate([sel_ind, [Last_ind]])
    return sel_ind
def plot_freq_distr(args):
    """
    Plot frequency distribution and the heatmap of the given training target and output.

    Args:
    args: A dictionary containing training target and output.

    Returns:
    None
    """
    y_fft = my_fft(args.training_target.detach().cpu().numpy()) / \
        args.training_size
    plt.figure()
    ax = plt.gca()
    plt.semilogy(y_fft+1e-5, label='Target')
    idx = SelectPeakIndex(y_fft, endpoint=False)
    plt.semilogy(idx, y_fft[idx]+1e-5, 'o')
    y_fft_pred = my_fft(args.training_output[-1])/args.training_size
    plt.semilogy(y_fft_pred+1e-5, label='Model output')
    plt.semilogy(idx, y_fft_pred[idx]+1e-5, 'o')
    plt.legend(fontsize=22)
    plt.xlabel('freq idx', fontsize=22)
    plt.ylabel('freq', fontsize=22)
    plt.gca().tick_params(axis='y', labelsize=22)
    plt.gca().tick_params(axis='x', labelsize=22)
    plt.tight_layout()
    plt.savefig(os.path.join(args.path, 'fft.png'), dpi=300)
    plt.show()
    plt.close()

    y_pred_epoch = np.squeeze(args.training_output)
    idx1 = idx[:3]
    abs_err = np.zeros([len(idx1), len(args.training_output)])
    y_fft = my_fft(args.training_target)
    tmp1 = y_fft[idx1]
    print(abs_err.shape)
    for i in range(len(y_pred_epoch)):
        tmp2 = my_fft(y_pred_epoch[i])[idx1]
        abs_err[:, i] = np.abs(tmp1 - tmp2)/(1e-5 + tmp1)



    plt.figure(figsize=(8, 6))
    ax = plt.gca()
    plt.pcolor(abs_err, cmap='RdBu', vmin=0.1, vmax=1, linewidths=0.4)
    plt.colorbar()


    plt.xlabel('Epoch', fontsize=22)

    # Set the y-axis ticks and labels to 1, 2, 3, and rotate the labels
    plt.yticks([0.5, 1.5, 2.5], [1, 2, 3], rotation=0, fontsize=22)

    # Set the y-axis tick parameters to hide the tick marks and set the tick label size
    plt.gca().yaxis.set_tick_params(size=0)
    plt.gca().tick_params(axis='y', labelsize=22)
    plt.gca().tick_params(axis='x', labelsize=22)


    plt.title('Absolute Error', fontsize=22)

    plt.tight_layout()

    plt.savefig(os.path.join(args.path, 'hot.png'))
    plt.show()
    plt.close()
def plot_target_pred_result(target,pred):
    """
    Plot the target.

    Args:
        target_tensor_tensor (torch.Tensor): The target tensor.
        pred_tensor (torch.Tensor): The prediction tensor.

    """
    plt.figure()
    ax = plt.gca()

    plt.plot(target.detach().cpu().numpy(),
             target.cpu().numpy(), 'b*', label='True')
    plt.plot(target.detach().cpu().numpy(),
             pred.cpu().numpy(), 'r-', label='Test')

    ax.tick_params(labelsize=18)
    plt.legend(fontsize=18)
    plt.show()
    print("The target function:")